In [1]:
import pandas as pd
import numpy as np
import datetime
import time
import itertools
from functools import wraps
from pathlib import Path

import plotly.graph_objects as go
import plotly.express as px
import dash
from dash import dcc
from dash import html
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages

import ibmdata

import ruptures as rpt

from concurrent.futures import ProcessPoolExecutor

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_columns', None)

%load_ext memory_profiler
from memory_profiler import profile

In [2]:
def timeit(method):
    @wraps(method)
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = method(*args, **kwargs)
        end_time = time.time()
        print(f"{method.__name__} => {(end_time-start_time)} s")

        return result

    return wrapper

In [3]:
class ChangePointDetector:
    def __init__(self):
        self.output_df = pd.DataFrame(columns=['parameter', 'lot_id', 'date'])
        self.data = None

    @timeit
    def pull_all_ilt_data(self, days_back=365, fc=('Q5', 'Q6', 'XQ', 'X2')):
        fc = str(fc)
        query = f"""
        SELECT LEFT(lot_Id,5) AS lot_id, family_Code, MIN(Last_test_date) AS date,
        tp.parm_Label as parameter, AVG(weighted_Mean) as mean 
        FROM DMIW.PTileWaferFact ptwf
        JOIN DMIW_SYSTEMS.TestParm tp ON tp.testparmkey = ptwf.testparmkey
        JOIN DMIW_SYSTEMS.TestedWafer tw ON tw.testedWaferKey = ptwf.testedWaferKey
        WHERE Last_test_date >= (current date - {days_back} days)
        AND Tech_id = '7HPP' 
        AND weighted_Mean is not null AND abs(weighted_Mean) < 1e25 
        AND family_Code IN '{fc}'
        GROUP BY LEFT(lot_Id,5), family_Code, tp.parm_Label
        ORDER BY MIN(Last_test_date), LEFT(lot_Id,5)
        """
        df = ibmdata.isdw.query(query)
        df['mean'] = df['mean'].astype(float).round(2)
        self.data = df
        print("Data Saved")

    def pull_ilt_data(self, parameter):
        return self.data[self.data.parameter == parameter].reset_index(drop=True)

    @staticmethod
    def change_point_plot(file_name, df, points, result, parm_name, cp_date, cp_lot):
        color_buffer = itertools.cycle(COLOR_CYCLE)

        fig, ax = plt.subplots(figsize=(10, 6))
        sns.set_theme(style="dark")
        sns.despine()
        x_labels = df[['lot_id', 'date']].astype('str').agg(' '.join, axis=1)

        num_points = len(points)
        plt.plot(range(num_points), points)
        plt.scatter(range(num_points), points)

        start = 0
        for stop, color in zip(result, color_buffer):
            left = max(0, start - 0.5)
            right = stop - 0.5
            plt.hlines(y=np.median(points[start:stop]),
                       xmin=left,
                       xmax=right,
                       color='r',
                       linestyle='--')
            plt.axvspan(left, right, facecolor=color, alpha=0.2, zorder=0)
            start = stop

        plt.title(f'Most recent change point for {parm_name} occurred on {cp_date} with lot {cp_lot}')
        plt.ylabel(parm_name)

        lower_bound = np.percentile(points, 25)
        upper_bound = np.percentile(points, 75)
        y_min = lower_bound - 1.5 * (upper_bound - lower_bound)
        y_max = upper_bound + 1.5 * (upper_bound - lower_bound)
        plt.ylim(y_min, y_max)
        plt.xlim(0, num_points - 1)
        plt.xticks(x_labels.index.values[result[:-1]], x_labels[result[:-1]], rotation=90, fontsize=12)

        plt.savefig(file_name, format='pdf', bbox_inches="tight", pad_inches=0.5)
        plt.close('all')
        return

    @staticmethod
    def change_point_detector(df, rbf_pen=10, time_delay=30, var='mean'):
        # Empty DataFrame
        if df.empty:
            # print('The df is empty')
            return
        if len(df) < 5:
            # print('Not enough data points')
            return

            # Find change point index

        points = np.array(df[var])
        model = "rbf"
        algor = rpt.Pelt(model=model).fit(points)
        result = algor.predict(pen=rbf_pen)

        if len(result) < 2:
            # print('No Change Points At All')
            return
        else:
            cp = result[-2]

        # find change point date
        parm_name = df.parameter[0]
        cp_lot = df.iloc[cp]['lot_id']
        cp_date = df.iloc[cp]['date']
        today = datetime.date.today()
        delta = datetime.timedelta(days=time_delay)
        if today - cp_date < delta:
            # print("Recent Change Point Detected")
            return {'df': df, 'points': points, 'result': result,
                    'parm_name': parm_name, 'cp_date': cp_date, 'cp_lot': cp_lot}
        else:
            # print("No Recent Change Points")
            return

    def param_cpd(self, param, rbf_pen=5, time_delay=30):

        # Pull data specific to param
        df = self.pull_ilt_data(param)

        # Run CPD algorithm
        return self.change_point_detector(df, time_delay=time_delay, rbf_pen=rbf_pen)

    @timeit
    def ilt_cpd_async(self, param_list, rbf_pen=5, time_delay=30):

        # Run analysis asynchronously
#         with ProcessPoolExecutor(max_workers=20) as executor:
#             return executor.map(self.param_cpd,
#                                 param_list,
#                                 itertools.repeat(rbf_pen),
#                                 itertools.repeat(time_delay),
#                                 timeout=60)
        return [self.param_cpd(param, rbf_pen, time_delay) for param in param_list]


In [4]:
levels_dict = {"M1": "M1", "M5": "M5", "MF": "LT"}
COLOR_CYCLE = ["#4286f4", "#f44174"]

"""
DAYS_BACK: Number of days to first data point
TIME_DELAY: Number of days to first change point
"""
DAYS_BACK = 365
TIME_DELAY = 200


@timeit
def run_cpd_async(levels, family_codes):
    today = datetime.datetime.now().strftime('%y%m%d')

    for family_code in family_codes:

        print(f"============================================\n{family_code}")

        # Create dir to save results; eg. --> ./CPDResults/220719/Q6
        dir_name = f"./CPDResults/{today}/{family_code}"
        Path(dir_name).mkdir(parents=True, exist_ok=True)

        # initialize cpd object
        model = ChangePointDetector()

        # Pull all data
        model.pull_all_ilt_data(DAYS_BACK, family_code)

        for level in levels:
            print(f"\n-------------------------------------------------\n\n{level}")

            # Read list of parameters from csv
            params = pd.read_csv(f'./Parms/{level}parms.csv', header=None, names=["Parameter", "Owner"])
            params["Parameter"] = params["Parameter"].astype('str') + f"_{levels_dict[level]}"

            # open pdf to save plots
            with PdfPages(f'{dir_name}/{family_code}_{level}_Graphs.pdf') as pdfFile:

                # run analysis on all parameters
                results = model.ilt_cpd_async(param_list=params["Parameter"],
                                              rbf_pen=5,
                                              time_delay=TIME_DELAY)

                results = list(filter(None, results))
                results = sorted(results, key=lambda x: x['cp_date'], reverse=True)

                for result in results:
                    # Plot result
                    model.change_point_plot(pdfFile, result['df'], result['points'], result['result'],
                                            result['parm_name'], result['cp_date'], result['cp_lot'])

                    # Save result to csv
                    model.output_df = model.output_df.append(pd.DataFrame([[result['parm_name'],
                                                                            result['cp_lot'],
                                                                            result['cp_date']]],
                                                                          columns=['parameter', 'lot_id', 'date']))

                # save results to csv
                model.output_df.merge(
                    params,
                    left_on='parameter',
                    right_on='Parameter'
                ).drop(
                    columns=['Parameter']
                ).rename(
                    columns={1: 'category'}
                ).to_csv(
                    f"{dir_name}/{family_code}_{level}_Data.csv",
                    index=False
                )
    return model

In [5]:
%reload_ext memory_profiler
%memit model = run_cpd_async(["M1", "M5", "MF"], ["Q6", "X2"])

Q6

Data Saved
pull_all_ilt_data => 1942.0337707996368 s

-------------------------------------------------

M1

ilt_cpd_async => 295.72152495384216 s

-------------------------------------------------

M5

ilt_cpd_async => 264.8623061180115 s

-------------------------------------------------

MF

ilt_cpd_async => 477.755428314209 s


/var/folders/r0/zl9449s534s95kg41vhmlkwm0000gn/T/ipykernel_3200/2225429511.py:62: UserWarning: Attempting to set identical bottom == top == 0.42 results in singular transformations; automatically expanding.
  plt.ylim(y_min, y_max)
/var/folders/r0/zl9449s534s95kg41vhmlkwm0000gn/T/ipykernel_3200/2225429511.py:62: UserWarning: Attempting to set identical bottom == top == -10.0 results in singular transformations; automatically expanding.
  plt.ylim(y_min, y_max)
/var/folders/r0/zl9449s534s95kg41vhmlkwm0000gn/T/ipykernel_3200/2225429511.py:62: UserWarning: Attempting to set identical bottom == top == 10.0 results in singular transformations; automatically expanding.
  plt.ylim(y_min, y_max)
/var/folders/r0/zl9449s534s95kg41vhmlkwm0000gn/T/ipykernel_3200/2225429511.py:62: UserWarning: Attempting to set identical bottom == top == -10.0 results in singular transformations; automatically expanding.
  plt.ylim(y_min, y_max)
/var/folders/r0/zl9449s534s95kg41vhmlkwm0000gn/T/ipykernel_3200/222542

X2

Data Saved
pull_all_ilt_data => 454.84683179855347 s

-------------------------------------------------

M1

ilt_cpd_async => 363.9081542491913 s

-------------------------------------------------

M5

ilt_cpd_async => 332.58514499664307 s


/var/folders/r0/zl9449s534s95kg41vhmlkwm0000gn/T/ipykernel_3200/2225429511.py:62: UserWarning: Attempting to set identical bottom == top == -10.0 results in singular transformations; automatically expanding.
  plt.ylim(y_min, y_max)
/var/folders/r0/zl9449s534s95kg41vhmlkwm0000gn/T/ipykernel_3200/2225429511.py:62: UserWarning: Attempting to set identical bottom == top == -10.0 results in singular transformations; automatically expanding.
  plt.ylim(y_min, y_max)
/var/folders/r0/zl9449s534s95kg41vhmlkwm0000gn/T/ipykernel_3200/2225429511.py:62: UserWarning: Attempting to set identical bottom == top == 10.0 results in singular transformations; automatically expanding.
  plt.ylim(y_min, y_max)
/var/folders/r0/zl9449s534s95kg41vhmlkwm0000gn/T/ipykernel_3200/2225429511.py:62: UserWarning: Attempting to set identical bottom == top == -10.0 results in singular transformations; automatically expanding.
  plt.ylim(y_min, y_max)
/var/folders/r0/zl9449s534s95kg41vhmlkwm0000gn/T/ipykernel_3200/22254


-------------------------------------------------

MF

ilt_cpd_async => 589.3347628116608 s


/var/folders/r0/zl9449s534s95kg41vhmlkwm0000gn/T/ipykernel_3200/2225429511.py:62: UserWarning: Attempting to set identical bottom == top == 0.66 results in singular transformations; automatically expanding.
  plt.ylim(y_min, y_max)
/var/folders/r0/zl9449s534s95kg41vhmlkwm0000gn/T/ipykernel_3200/2225429511.py:62: UserWarning: Attempting to set identical bottom == top == 0.23 results in singular transformations; automatically expanding.
  plt.ylim(y_min, y_max)


run_cpd_async => 4879.371462106705 s
peak memory: 4487.09 MiB, increment: 4292.48 MiB


In [ ]:
model.outputdf